In [2]:
import scoring_functions as fn

#### Operation ID @ SLM prompt with extra examples

How well different models generate an operation ID for use in an OpenAPI spec, given an API request

In [3]:
scenario = "Generate OpenAPI operation ID using SLM prompt w/extra examples"
split_response = True
prompt_template = """
    **Prompt:**
    Generate an operation ID for an OpenAPI specification based on the HTTP method and URL provided. Follow these rules:
    - The operation ID should be in camelCase format.
    - Start with a verb that matches the HTTP method (e.g., `get`, `create`, `update`, `delete`).
    - Use descriptive words from the URL path.
    - Replace path parameters (e.g., `{{userId}}`) with relevant nouns in singular form (e.g., `User`).
    - Do not provide explanations or any other text; respond only with the operation ID.

    Example:
    **Request:** `GET https://api.contoso.com/books/{{books-id}}`
    getBook

    Example:
    **Request:** `GET https://api.contoso.com/books/{{books-id}}/authors`
    getBookAuthors

    Example:
    **Request:** `GET https://api.contoso.com/books/{{books-id}}/authors/{{authors-id}}`
    getBookAuthor

    Example:
    **Request:** `POST https://api.contoso.com/books/{{books-id}}/authors`
    addBookAuthor

    Now, generate the operation ID for the following:
    **Request:** `{request}`
    """.strip()
tasks = [
    {
        "name": "getCalendarsForUser",
        "reference_texts": [
            "getCalendarsForUser",
            "listUserCalendars",
            "getUserCalendars",
            "fetchUserCalendars",
            "listCalendarsForUser"
        ],
        "prompt": prompt_template.format(request="GET https://graph.microsoft.com/users/{{users-id}}/calendars")
    },
    {
        "name": "getUser",
        "reference_texts": [
            "getUser",
            "fetchUser",
            "retrieveUser"
        ],
        "prompt": prompt_template.format(request="GET https://graph.microsoft.com/users/{{users-id}}")
    },
    {
        "name": "getCalendarForUser",
        "reference_texts": [
            "getCalendarForUser",
            "fetchUserCalendar",
            "getUserCalendar"
        ],
        "prompt": prompt_template.format(request="GET https://graph.microsoft.com/users/{{users-id}}/calendars/{{calendars-id}}")
    },
    {
        "name": "postCalendarForUser",
        "reference_texts": [
            "postCalendarForUser"
            "createUserCalendar",
            "createCalendarForUser",
            "addUserCalendar",
            "postUserCalendar",
            "addCalendarForUser"
        ],
        "prompt": prompt_template.format(request="POST https://graph.microsoft.com/users/{{users-id}}/calendars")
    }
]
score_threshold = 0.8
metric_scorers = [
    {
        'name': 'BERT-F',
        'fn': fn.calculate_bert_f,
        'weight': 0.45
    },
    {
        'name': 'Edit distance',
        'fn': fn.calculate_edit_distance,
        'weight': 0.1
    },
    {
        'name': 'ROUGE-1',
        'fn': fn.calculate_rouge1,
        'weight': 0.25
    },
    {
        'name': 'ROUGE-L',
        'fn': fn.calculate_rougeL,
        'weight': 0.2
    }
]